In [74]:
import numpy as np

In [ ]:
def get_error(x, y):
    error = x - y
    print( np.linalg.norm(error.flatten()) )

In [75]:
from pyscf import gto, scf

from ccpy.models.calculation import Calculation
from ccpy.interfaces.pyscf_tools import load_pyscf_integrals
from ccpy.drivers.driver import cc_driver

mol = gto.Mole()
mol.build(
    atom="""F 0.0 0.0 -2.66816
            F 0.0 0.0  2.66816""",
    basis="ccpvdz",
    charge=0,
    spin=0,
    symmetry="D2H",
    cart=True,
    unit='Bohr',
)
mf = scf.ROHF(mol)
mf.kernel()


system, H0 = load_pyscf_integrals(mf, nfrozen=2)
system.print_info()

converged SCF energy = -198.420096282673
   System Information:
   ----------------------------------------------------
     Number of correlated electrons = 14
     Number of correlated orbitals = 28
     Number of frozen orbitals = 2
     Number of alpha occupied orbitals = 7
     Number of alpha unoccupied orbitals = 21
     Number of beta occupied orbitals = 7
     Number of beta unoccupied orbitals = 21
     Charge = 0
     Point group = D2h
     Symmetry of reference = AG
     Spin multiplicity of reference = 1

      MO #        Energy (a.u.)      Symmetry    Occupation
-----------------------------------------------------------
         1           -26.443066            AG           2.0
         2           -26.443033           B1U           2.0
         3            -1.597219            AG           2.0
         4            -1.589112           B1U           2.0
         5            -0.725614           B2U           2.0
         6            -0.725614           B3U           

In [76]:
calculation = Calculation(
    order=3,
    active_orders=[None],
    num_active=[None],
    calculation_type="ccsdt",
    convergence_tolerance=1.0e-08
)

T0, total_energy, is_converged = cc_driver(calculation, system, H0)

   --------------------------------------------------
   Calculation type =  CCSDT
   Maximum iterations = 60
   Convergence tolerance = 1e-08
   Energy shift = 0.0
   DIIS size =  6
   RHF symmetry = False
   --------------------------------------------------

   CC calculation started at 2022-03-09 12:02:00 

   Energy of initial guess =         0.0000000000

                 Iter.         Residuum                 δE                  ΔE               Wall time
-----------------------------------------------------------------------------------------------------
                   0         0.7950811421        -0.5760307223        -0.5760307223          (0.0m 6.9s)
                   1         0.2600010113        -0.0470790932        -0.6231098155          (0.0m 7.6s)
                   2         0.0673753279        -0.0073211685        -0.6304309840          (0.0m 7.6s)
                   3         0.0268057665        -0.0056287055        -0.6360596895          (0.0m 7.1s)
           

In [77]:
from ccpy.hbar.hbar_ccsd import build_hbar_ccsd

H = build_hbar_ccsd(T0, H0)

In [78]:
noccupied_alpha = T0.a.shape[1]
nunoccupied_alpha = T0.a.shape[0]
noccupied_beta = T0.b.shape[1]
nunoccupied_beta = T0.b.shape[0]

nact_occupied_alpha = 2
nact_unoccupied_alpha = 3
nact_occupied_beta = 2
nact_unoccupied_beta = 3

nunact_occupied_alpha = noccupied_alpha - nact_occupied_alpha
nunact_unoccupied_alpha = nunoccupied_alpha - nact_unoccupied_alpha
nunact_occupied_beta = noccupied_beta - nact_occupied_beta
nunact_unoccupied_beta = nunoccupied_beta - nact_unoccupied_beta

oa = slice(0, noccupied_alpha - nact_occupied_alpha)
Oa = slice(noccupied_alpha - nact_occupied_alpha, noccupied_alpha)
va = slice(nact_unoccupied_alpha, nunoccupied_alpha)
Va = slice(0, nact_unoccupied_alpha)

ob = slice(0, noccupied_beta - nact_occupied_beta)
Ob = slice(noccupied_beta - nact_occupied_beta, noccupied_beta)
vb = slice(nact_unoccupied_beta, nunoccupied_beta)
Vb = slice(0, nact_unoccupied_beta)

shift = 0.0

In [79]:
# An additional step needs to be done. We need to zero all T3 amplitudes 
# outside of the active space before computing the full CCSDT updates.
# This is because the active-space CC updates assume that T3 is 0 outside of
# the active space. If this is not true in the full scheme, then the results
# will not match.

In [103]:
# Populating the active-space structure
from ccpy.models.operators import ClusterOperator

T = ClusterOperator(system, 3, active_orders=[3], num_active=[1])
T.aaa.VVVOOO = T0.aaa[Va, Va, Va, Oa, Oa, Oa]
T.aaa.VVvOOO = T0.aaa[Va, Va, va, Oa, Oa, Oa]
T.aaa.VVVoOO = T0.aaa[Va, Va, Va, oa, Oa, Oa]
T.aaa.VVvoOO = T0.aaa[Va, Va, va, oa, Oa, Oa]
T.aaa.VvvOOO = T0.aaa[Va, va, va, Oa, Oa, Oa]
T.aaa.VVVooO = T0.aaa[Va, Va, Va, oa, oa, Oa]
T.aaa.VVvooO = T0.aaa[Va, Va, va, oa, oa, Oa]
T.aaa.VvvoOO = T0.aaa[Va, va, va, oa, Oa, Oa]
T.aaa.VvvooO = T0.aaa[Va, va, va, oa, oa, Oa]

T.aaa.VVVOOO = T0.aaa[Va, Va, Va, Oa, Oa, Oa]
T.aaa.VVvOOO = T0.aaa[Va, Va, va, Oa, Oa, Oa]
T.aaa.VVVoOO = T0.aaa[Va, Va, Va, oa, Oa, Oa]
T.aaa.VVvoOO = T0.aaa[Va, Va, va, oa, Oa, Oa]
T.aaa.VvvOOO = T0.aaa[Va, va, va, Oa, Oa, Oa]
T.aaa.VVVooO = T0.aaa[Va, Va, Va, oa, oa, Oa]
T.aaa.VVvooO = T0.aaa[Va, Va, va, oa, oa, Oa]
T.aaa.VvvoOO = T0.aaa[Va, va, va, oa, Oa, Oa]
T.aaa.VvvooO = T0.aaa[Va, va, va, oa, oa, Oa]

# T.aab
T.aab.VVVOOO = T0.aab[Va,Va,Vb,Oa,Oa,Ob]
T.aab.VVVoOO = T0.aab[Va,Va,Vb,oa,Oa,Ob]
T.aab.VVVOOo = T0.aab[Va,Va,Vb,Oa,Oa,ob]
T.aab.VVVoOo = T0.aab[Va,Va,Vb,oa,Oa,ob]
T.aab.VVVooO = T0.aab[Va,Va,Vb,oa,oa,Ob]
T.aab.VVVooo = T0.aab[Va,Va,Vb,oa,oa,ob]

T.aab.VvVOOO = T0.aab[Va,va,Vb,Oa,Oa,Ob]
T.aab.VvVoOO = T0.aab[Va,va,Vb,oa,Oa,Ob]
T.aab.VvVOOo = T0.aab[Va,va,Vb,Oa,Oa,ob]
T.aab.VvVoOo = T0.aab[Va,va,Vb,oa,Oa,ob]
T.aab.VvVooO = T0.aab[Va,va,Vb,oa,oa,Ob]
T.aab.VvVooo = T0.aab[Va,va,Vb,oa,oa,ob]

T.aab.VVvOOO = T0.aab[Va,Va,vb,Oa,Oa,Ob]
T.aab.VVvoOO = T0.aab[Va,Va,vb,oa,Oa,Ob]
T.aab.VVvOOo = T0.aab[Va,Va,vb,Oa,Oa,ob]
T.aab.VVvoOo = T0.aab[Va,Va,vb,oa,Oa,ob]
T.aab.VVvooO = T0.aab[Va,Va,vb,oa,oa,Ob]
T.aab.VVvooo = T0.aab[Va,Va,vb,oa,oa,ob]

T.aab.VvvOOO = T0.aab[Va,va,vb,Oa,Oa,Ob]
T.aab.VvvoOO = T0.aab[Va,va,vb,oa,Oa,Ob]
T.aab.VvvOOo = T0.aab[Va,va,vb,Oa,Oa,ob]
T.aab.VvvoOo = T0.aab[Va,va,vb,oa,Oa,ob]
T.aab.VvvooO = T0.aab[Va,va,vb,oa,oa,Ob]
T.aab.Vvvooo = T0.aab[Va,va,vb,oa,oa,ob]

T.aab.vvVOOO = T0.aab[va,va,Vb,Oa,Oa,Ob]
T.aab.vvVoOO = T0.aab[va,va,Vb,oa,Oa,Ob]
T.aab.vvVOOo = T0.aab[va,va,Vb,Oa,Oa,ob]
T.aab.vvVoOo = T0.aab[va,va,Vb,oa,Oa,ob]
T.aab.vvVooO = T0.aab[va,va,Vb,oa,oa,Ob]
T.aab.vvVooo = T0.aab[va,va,Vb,oa,oa,ob]

T.aab.vvvOOO = T0.aab[va,va,vb,Oa,Oa,Ob]
T.aab.vvvoOO = T0.aab[va,va,vb,oa,Oa,Ob]
T.aab.vvvOOo = T0.aab[va,va,vb,Oa,Oa,ob]
T.aab.vvvoOo = T0.aab[va,va,vb,oa,Oa,ob]
T.aab.vvvooO = T0.aab[va,va,vb,oa,oa,Ob]
T.aab.vvvooo = T0.aab[va,va,vb,oa,oa,ob]

In [104]:
from ccpy.utilities.updates import cc_loops2
from ccpy.utilities.updates import cc_active_loops

In [108]:
# The issue here is that the diagram with an extra weight factor is not actually equal to the
# underlying diagram with the correct antisymmetrizer applied. They are only equal once ALL
# antisymmetrizers are applied to the former diagram. That is the point of the weight factor! 
# As a result, you won't see equivalence until you apply the full antisymmetrizer within the 
# update loop.

# Diagram 1:
D1 = -np.einsum("mk,abcijm->abcijk", H.a.oo, T0.aaa, optimize=True)
D1 -= np.transpose(D1, (0, 1, 2, 5, 4, 3)) + np.transpose(D1, (0, 1, 2, 3, 5, 4))

D2 = np.einsum('ae,ebcijk->abcijk', H.a.vv, T0.aaa, optimize=True)
D2 -= np.transpose(D2, (1, 0, 2, 3, 4, 5)) + np.transpose(D2, (2, 1, 0, 3, 4, 5))

D3 = 0.5 * np.einsum('mnij,abcmnk->abcijk', H.aa.oooo, T0.aaa, optimize=True)
D3 -= np.transpose(D3, (0, 1, 2, 5, 4, 3)) + np.transpose(D3, (0, 1, 2, 3, 5, 4))

D4 = 0.5 * np.einsum('abef,efcijk->abcijk', H.aa.vvvv, T0.aaa, optimize=True)
D4 -= np.transpose(D4, (2, 1, 0, 3, 4, 5)) + np.transpose(D4, (0, 2, 1, 3, 4, 5))

D5 = np.einsum('amie,ebcmjk->abcijk', H.aa.voov, T0.aaa, optimize=True)
D5 -= np.transpose(D5, (1, 0, 2, 3, 4, 5)) + np.transpose(D5, (2, 1, 0, 3, 4, 5))
D5 -= np.transpose(D5, (0, 1, 2, 4, 3, 5)) + np.transpose(D5, (0, 1, 2, 5, 4, 3))

D6 = np.einsum('amie,bcejkm->abcijk', H.ab.voov, T0.aab, optimize=True)
D6 -= np.transpose(D6, (1, 0, 2, 3, 4, 5)) + np.transpose(D6, (2, 1, 0, 3, 4, 5))
D6 -= np.transpose(D6, (0, 1, 2, 4, 3, 5)) + np.transpose(D6, (0, 1, 2, 5, 4, 3))

X3A = D1 + D2 + D3 + D4 + D5 + D6

## Projection 1

In [109]:
D1_proj1 = (
        +1.0 * np.einsum('mI,CBAmJK->ABCIJK', H.a.oo[oa, Oa], T.aaa.VVVoOO, optimize=True)
        + 1.0 * np.einsum('MI,CBAMJK->ABCIJK', H.a.oo[Oa, Oa], T.aaa.VVVOOO, optimize=True)
)
D1_proj1 -= np.transpose(D1_proj1, (0, 1, 2, 5, 4, 3)) + np.transpose(D1_proj1, (0, 1, 2, 4, 3, 5))

D2_proj1 = (
        -1.0*np.einsum('Ae,CBeIJK->ABCIJK', H.a.vv[Va, va], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('AE,CBEIJK->ABCIJK', H.a.vv[Va, Va], T.aaa.VVVOOO, optimize=True)
)
D2_proj1 -= np.transpose(D2_proj1, (1, 0, 2, 3, 4, 5)) + np.transpose(D2_proj1, (2, 1, 0, 3, 4, 5))

D3_proj1 = (
        -0.5*np.einsum('mnIJ,CBAmnK->ABCIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('mNIJ,CBAmNK->ABCIJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aaa.VVVoOO, optimize=True)
        -0.5*np.einsum('MNIJ,CBAMNK->ABCIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aaa.VVVOOO, optimize=True)
)
D3_proj1 -= np.transpose(D3_proj1, (0, 1, 2, 5, 4, 3)) + np.transpose(D3_proj1, (0, 1, 2, 3, 5, 4))

D4_proj1 = (
        -0.5*np.einsum('ABef,CfeIJK->ABCIJK', H.aa.vvvv[Va, Va, va, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('ABeF,CFeIJK->ABCIJK', H.aa.vvvv[Va, Va, va, Va], T.aaa.VVvOOO, optimize=True)
        -0.5*np.einsum('ABEF,CFEIJK->ABCIJK', H.aa.vvvv[Va, Va, Va, Va], T.aaa.VVVOOO, optimize=True)
)
D4_proj1 -= np.transpose(D4_proj1, (2, 1, 0, 3, 4, 5)) + np.transpose(D4_proj1, (0, 2, 1, 3, 4, 5))

D5_proj1 = (
        -1.0*np.einsum('AmIe,CBemJK->ABCIJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('AMIe,CBeMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('AmIE,CBEmJK->ABCIJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('AMIE,CBEMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVVOOO, optimize=True)
)
D5_proj1 -= np.transpose(D5_proj1, (1, 0, 2, 3, 4, 5)) + np.transpose(D5_proj1, (2, 1, 0, 3, 4, 5))
D5_proj1 -= np.transpose(D5_proj1, (0, 1, 2, 4, 3, 5)) + np.transpose(D5_proj1, (0, 1, 2, 5, 4, 3))

D6_proj1 = (
        -1.0*np.einsum('AmIe,CBeJKm->ABCIJK', H.ab.voov[Va, ob, Oa, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('AmIE,CBEJKm->ABCIJK', H.ab.voov[Va, ob, Oa, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('AMIe,CBeJKM->ABCIJK', H.ab.voov[Va, Ob, Oa, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('AMIE,CBEJKM->ABCIJK', H.ab.voov[Va, Ob, Oa, Vb], T.aab.VVVOOO, optimize=True)
)
D6_proj1 -= np.transpose(D6_proj1, (1, 0, 2, 3, 4, 5)) + np.transpose(D6_proj1, (2, 1, 0, 3, 4, 5))
D6_proj1 -= np.transpose(D6_proj1, (0, 1, 2, 4, 3, 5)) + np.transpose(D6_proj1, (0, 1, 2, 5, 4, 3))

X3A_proj1 = D1_proj1 + D2_proj1 + D3_proj1 + D4_proj1 + D5_proj1 + D6_proj1

get_error(X3A[Va, Va, Va, Oa, Oa, Oa], X3A_proj1)


0.0


### Projection 2

In [67]:
D1_proj2 = (
        +1.0 * np.einsum('mI,BAcmJK->ABcIJK', Hbar.a.oo[oa, Oa], T.aaa.VVvoOO, optimize=True)
        + 1.0 * np.einsum('MI,BAcMJK->ABcIJK', Hbar.a.oo[Oa, Oa], T.aaa.VVvOOO, optimize=True)
)
D1_proj2 -= np.transpose(D1_proj2, (0, 1, 2, 5, 4, 3)) + np.transpose(D1_proj2, (0, 1, 2, 4, 3, 5))

x1 =  (
        +1.0*np.einsum('AmIe,BcemJK->ABcIJK', H.aa.voov[Va, oa, Oa, va], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('AmIE,BEcmJK->ABcIJK', H.aa.voov[Va, oa, Oa, Va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('AMIe,BceMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, va], T.aaa.VvvOOO, optimize=True)
        -1.0*np.einsum('AMIE,BEcMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, Va], T.aaa.VVvOOO, optimize=True)
)
x1 -= np.transpose(x1, (1, 0, 2, 3, 4, 5))
x1 -= np.transpose(x1, (0, 1, 2, 4, 3, 5)) + np.transpose(x1, (0, 1, 2, 5, 4, 3))
x2 = (
        +1.0*np.einsum('cmIe,ABemJK->ABcIJK', H.aa.voov[va, oa, Oa, va], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('cmIE,ABEmJK->ABcIJK', H.aa.voov[va, oa, Oa, Va], T.aaa.VVVoOO, optimize=True)
        +1.0*np.einsum('cMIe,ABeMJK->ABcIJK', H.aa.voov[va, Oa, Oa, va], T.aaa.VVvOOO, optimize=True)
        +1.0*np.einsum('cMIE,ABEMJK->ABcIJK', H.aa.voov[va, Oa, Oa, Va], T.aaa.VVVOOO, optimize=True)
)
x2 -= np.transpose(x2, (0, 1, 2, 4, 3, 5)) + np.transpose(x2, (0, 1, 2, 5, 4, 3))
D5_proj2 = x1 + x2

X3A_proj2 = D1_proj2 + D5_proj2

get_error(X3A[Va, Va, va, Oa, Oa, Oa], X3A_proj2)

0.0


### Projection 3

In [59]:
x1 = (
        +1.0 * np.einsum('mi,CBAmJK->ABCiJK', Hbar.a.oo[oa, oa], T.aaa.VVVoOO, optimize=True)
        + 1.0 * np.einsum('Mi,CBAMJK->ABCiJK', Hbar.a.oo[Oa, oa], T.aaa.VVVOOO, optimize=True)
)
x2 = (
        -1.0 * np.einsum('mJ,CBAmiK->ABCiJK', Hbar.a.oo[oa, Oa], T.aaa.VVVooO, optimize=True)
        + 1.0 * np.einsum('MJ,CBAiMK->ABCiJK', Hbar.a.oo[Oa, Oa], T.aaa.VVVoOO, optimize=True)
)
x2 -= np.transpose(x2, (0, 1, 2, 3, 5, 4))
D1_proj3 = x1 + x2
get_error(D1[Va, Va, Va, oa, Oa, Oa], D1_proj3)

0.0


### Projection 4

In [60]:
x1 = (
        +1.0 * np.einsum('mi,BAcmJK->ABciJK', Hbar.a.oo[oa, oa], T.aaa.VVvoOO, optimize=True)
        + 1.0 * np.einsum('Mi,BAcMJK->ABciJK', Hbar.a.oo[Oa, oa], T.aaa.VVvOOO, optimize=True)
)
x2 = (
        -1.0 * np.einsum('mJ,BAcmiK->ABciJK', Hbar.a.oo[oa, Oa], T.aaa.VVvooO, optimize=True)
        + 1.0 * np.einsum('MJ,BAciMK->ABciJK', Hbar.a.oo[Oa, Oa], T.aaa.VVvoOO, optimize=True)
)
x2 -= np.transpose(x2, (0, 1, 2, 3, 5, 4))
D1_proj4 = x1 + x2
get_error(D1[Va, Va, va, oa, Oa, Oa], D1_proj4)

2.9162546041251223e-19


### Projection 5

In [61]:

D1_proj5 = (
        +1.0 * np.einsum('mI,AcbmJK->AbcIJK', H.a.oo[oa, Oa], T.aaa.VvvoOO, optimize=True)
        + 1.0 * np.einsum('MI,AcbMJK->AbcIJK', H.a.oo[Oa, Oa], T.aaa.VvvOOO, optimize=True)
)
D1_proj5 -= np.transpose(D1_proj5, (0, 1, 2, 4, 3, 5)) + np.transpose(D1_proj5, (0, 1, 2, 5, 4, 3))
get_error(D1[Va, va, va, Oa, Oa, Oa], D1_proj5)


0.0


### Projection 6

In [ ]:
# D1_proj6 = (2.0 / 12.0) * (
#         +1.0 * np.einsum('mi,CBAmjK->ABCijK', H.a.oo[oa, oa], T.aaa.VVVooO, optimize=True)
#         - 1.0 * np.einsum('Mi,CBAjMK->ABCijK', H.a.oo[Oa, oa], T.aaa.VVVoOO, optimize=True)
# )
# get_error(D1[Va, Va, Va, oa, oa, Oa], D1_proj6)


### Projection 7

In [ ]:
# D1_proj7 = (2.0 / 4.0) * (
#         +1.0 * np.einsum('mi,BAcmjK->ABcijK', H.a.oo[oa, oa], T.aaa.VVvooO, optimize=True)
#         - 1.0 * np.einsum('Mi,BAcjMK->ABcijK', H.a.oo[Oa, oa], T.aaa.VVvoOO, optimize=True)
# )
# get_error(D1[Va, Va, va, oa, oa, Oa], D1_proj7)

### Projection 8

In [ ]:
# D1_proj8 = (1.0 / 4.0) * (
#         +1.0 * np.einsum('mi,AcbmJK->AbciJK', H.a.oo[oa, oa], T.aaa.VvvoOO, optimize=True)
#         + 1.0 * np.einsum('Mi,AcbMJK->AbciJK', H.a.oo[Oa, oa], T.aaa.VvvOOO, optimize=True)
# )
# get_error(D1[Va, va, va, oa, Oa, Oa], D1_proj8)


### Projection 9

In [ ]:
# D1_proj9 = (2.0 / 4.0) * (
#         +1.0 * np.einsum('mi,AcbmjK->AbcijK', H.a.oo[oa, oa], T.aaa.VvvooO, optimize=True)
#         - 1.0 * np.einsum('Mi,AcbjMK->AbcijK', H.a.oo[Oa, oa], T.aaa.VvvoOO, optimize=True)
# )
# get_error(D1[Va, va, va, oa, oa, Oa], D1_proj9)